1. Obtención de Datos
• Extrae datasets desde al menos dos fuentes distintas (CSV, Excel,
JSON o API REST usando requests).
• Carga los datos utilizando pandas, verificando su correcta lectura y
estructura.


In [2]:
import pandas as pd
import numpy as np

#Lectura de archivo elegido usando metodo read_csv
df = pd.read_csv("personality_dataset.csv")


In [3]:
#Lectura de datos contenidos en Excel
df_excel = pd.read_excel('dataset_generado.xlsx')
print(df_excel)

         Genero Tiene_mascota
0      Femenino            Si
1           NaN            No
2     Masculino            Si
3     Masculino            Si
4      Femenino            No
...         ...           ...
2895  Masculino            No
2896  Masculino            Si
2897   Femenino           NaN
2898   Femenino           NaN
2899   Femenino            Si

[2900 rows x 2 columns]


2. Exploración Inicial
• Utiliza métodos básicos (head(), info(), describe(), value_counts(),
etc.) para explorar el set de datos y detectar posibles problemas.
• Aplica filtros y selecciones con loc[] o iloc[] para identificar
registros específicos.

In [4]:
df.head()
#print(df)

print(df.head())
print(df.describe())
print(df.info())
print(df["Personality"].value_counts())
print(df["Personality"].unique())

#Usamos el filtraje con [loc] para filtrar por personalidad y luego sacamos el promedio de vcalores numericos asociados a cada tipo:

introverts = df.loc[df['Personality'] == 'Introvert']
extroverts = df.loc[df['Personality'] == 'Extrovert']

comparación = pd.DataFrame({
    'Introvert': introverts.mean(numeric_only=True),
    'Extrovert': extroverts.mean(numeric_only=True)
})

display(comparación)



   Time_spent_Alone Stage_fear  Social_event_attendance  Going_outside  \
0               4.0         No                      4.0            6.0   
1               9.0        Yes                      0.0            0.0   
2               9.0        Yes                      1.0            2.0   
3               0.0         No                      6.0            7.0   
4               3.0         No                      9.0            4.0   

  Drained_after_socializing  Friends_circle_size  Post_frequency Personality  
0                        No                 13.0             5.0   Extrovert  
1                       Yes                  0.0             3.0   Introvert  
2                       Yes                  5.0             2.0   Introvert  
3                        No                 14.0             8.0   Extrovert  
4                        No                  8.0             5.0   Extrovert  
       Time_spent_Alone  Social_event_attendance  Going_outside  \
count       28

,Introvert,Extrovert
Time_spent_Alone,7.080435,2.067261
Social_event_attendance,1.778909,6.016405
Going_outside,1.272859,4.634615
Friends_circle_size,3.196793,9.173673
Post_frequency,1.368918,5.638546


3. Limpieza y Tratamiento de Valores Faltantes
• Identifica valores nulos o inconsistentes y aplica criterios de
imputación (por ejemplo: eliminar, reemplazar por
media/mediana/moda o crear indicadores).
• Justifica brevemente la estrategia utilizada.

In [7]:
#Agrego columna de ID a ambas tablas para poder unirlas despues por este valor
df['ID'] = range(1, len(df) + 1)
df.set_index('ID', inplace=True)

df_excel['ID'] = range(1, len(df) + 1)
df_excel.set_index('ID', inplace=True)

print(df.head())
print(df_excel.head())

    Time_spent_Alone Stage_fear  Social_event_attendance  Going_outside  \
ID                                                                        
1                4.0         No                      4.0            6.0   
2                9.0        Yes                      0.0            0.0   
3                9.0        Yes                      1.0            2.0   
4                0.0         No                      6.0            7.0   
5                3.0         No                      9.0            4.0   

   Drained_after_socializing  Friends_circle_size  Post_frequency Personality  
ID                                                                             
1                         No                 13.0             5.0   Extrovert  
2                        Yes                  0.0             3.0   Introvert  
3                        Yes                  5.0             2.0   Introvert  
4                         No                 14.0             8.0   Extrov

In [9]:
#Aqui hago el merge de los datos de ambas fuentes
df_completo = pd.merge(df, df_excel, on="ID", how="inner")

print(df_completo)

      Time_spent_Alone Stage_fear  Social_event_attendance  Going_outside  \
ID                                                                          
1                  4.0         No                      4.0            6.0   
2                  9.0        Yes                      0.0            0.0   
3                  9.0        Yes                      1.0            2.0   
4                  0.0         No                      6.0            7.0   
5                  3.0         No                      9.0            4.0   
...                ...        ...                      ...            ...   
2896               3.0         No                      7.0            6.0   
2897               3.0         No                      8.0            3.0   
2898               4.0        Yes                      1.0            1.0   
2899              11.0        Yes                      1.0            NaN   
2900               3.0         No                      6.0            6.0   

In [11]:
# Identificar nulos

print("Vamos a evaluar valores nulos")
print(df_completo.isnull().sum())

# Reemplazar nulos en columnas numéricas con valores promedios
numericas = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']
for col in numericas:
  df_completo[col].fillna(df_completo[col].mean(), inplace=True)

categóricas = ['Stage_fear', 'Drained_after_socializing', 'Personality']
for col in categóricas:
    df_completo[col] = df_completo[col].fillna(df_completo[col].mode()[0])

df_completo

Vamos a evaluar valores nulos
Time_spent_Alone               0
Stage_fear                     0
Social_event_attendance        0
Going_outside                  0
Drained_after_socializing      0
Friends_circle_size            0
Post_frequency                 0
Personality                    0
Genero                       287
Tiene_mascota                146
dtype: int64


/tmp/ipython-input-11-1429910273.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_completo[col].fillna(df_completo[col].mean(), inplace=True)


,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality,Genero,Tiene_mascota
ID,,,,,,,,,,
1,4.0,No,4.0,6.0,No,13.0,5.0,Extrovert,Femenino,Si
2,9.0,Yes,0.0,0.0,Yes,0.0,3.0,Introvert,NaN,No
3,9.0,Yes,1.0,2.0,Yes,5.0,2.0,Introvert,Masculino,Si
4,0.0,No,6.0,7.0,No,14.0,8.0,Extrovert,Masculino,Si
5,3.0,No,9.0,4.0,No,8.0,5.0,Extrovert,Femenino,No
...,...,...,...,...,...,...,...,...,...,...
2896,3.0,No,7.0,6.0,No,6.0,6.0,Extrovert,Masculino,No
2897,3.0,No,8.0,3.0,No,14.0,9.0,Extrovert,Masculino,Si
2898,4.0,Yes,1.0,1.0,Yes,4.0,0.0,Introvert,Femenino,NaN


In [12]:
#Uso metodo de agrupación para revisar datos especificos por promedio de columna Friends_circle-size:

agrupado = df_completo.groupby('Personality')['Friends_circle_size'].mean()
print(agrupado)

Personality
Extrovert    9.095744
Introvert    3.277465
Name: Friends_circle_size, dtype: float64


• Realiza transformaciones estructurales usando pandas: renombrar
columnas, cambiar tipos de datos, crear nuevas variables a partir
de otras, etc.

In [15]:
# Cambiar nombres largos por nombres más simples
df_completo.rename(columns={
    'Time_spent_Alone': 'Tiempo_Solo',
    'Friends_circle_size': 'Numero_Amigos',
    'Social_event_attendance': 'Salidas_sociales',
    'Drained_after_socializing': 'Agotamiento_social'
}, inplace=True)

df_completo

,Tiempo_Solo,Stage_fear,Salidas_sociales,Going_outside,Agotamiento_social,Numero_Amigos,Post_frequency,Personality,Genero,Tiene_mascota
ID,,,,,,,,,,
1,4.0,No,4.0,6.0,No,13.0,5.0,Extrovert,Femenino,Si
2,9.0,Yes,0.0,0.0,Yes,0.0,3.0,Introvert,NaN,No
3,9.0,Yes,1.0,2.0,Yes,5.0,2.0,Introvert,Masculino,Si
4,0.0,No,6.0,7.0,No,14.0,8.0,Extrovert,Masculino,Si
5,3.0,No,9.0,4.0,No,8.0,5.0,Extrovert,Femenino,No
...,...,...,...,...,...,...,...,...,...,...
2896,3.0,No,7.0,6.0,No,6.0,6.0,Extrovert,Masculino,No
2897,3.0,No,8.0,3.0,No,14.0,9.0,Extrovert,Masculino,Si
2898,4.0,Yes,1.0,1.0,Yes,4.0,0.0,Introvert,Femenino,NaN


In [16]:
# Ver tipos actuales

df_completo['Post_frequency'] = df_completo['Post_frequency'].astype('int64')

print(df_completo.dtypes)

Tiempo_Solo           float64
Stage_fear             object
Salidas_sociales      float64
Going_outside         float64
Agotamiento_social     object
Numero_Amigos         float64
Post_frequency          int64
Personality            object
Genero                 object
Tiene_mascota          object
dtype: object


In [17]:
#Agrego una columna nueva evaluando la columna de Numero_Amigos
df_completo['Etiqueta_Amigos'] = df_completo['Numero_Amigos'].apply(lambda x: 'Pocos' if x < 5 else 'Muchos')


df_completo

,Tiempo_Solo,Stage_fear,Salidas_sociales,Going_outside,Agotamiento_social,Numero_Amigos,Post_frequency,Personality,Genero,Tiene_mascota,Etiqueta_Amigos
ID,,,,,,,,,,,
1,4.0,No,4.0,6.0,No,13.0,5,Extrovert,Femenino,Si,Muchos
2,9.0,Yes,0.0,0.0,Yes,0.0,3,Introvert,NaN,No,Pocos
3,9.0,Yes,1.0,2.0,Yes,5.0,2,Introvert,Masculino,Si,Muchos
4,0.0,No,6.0,7.0,No,14.0,8,Extrovert,Masculino,Si,Muchos
5,3.0,No,9.0,4.0,No,8.0,5,Extrovert,Femenino,No,Muchos
...,...,...,...,...,...,...,...,...,...,...,...
2896,3.0,No,7.0,6.0,No,6.0,6,Extrovert,Masculino,No,Muchos
2897,3.0,No,8.0,3.0,No,14.0,9,Extrovert,Masculino,Si,Muchos
2898,4.0,Yes,1.0,1.0,Yes,4.0,0,Introvert,Femenino,NaN,Pocos


Voy a crear un dataset adicional en excel para poder agregarlo y cumplir con la condición de multiples fuentes de datos:

In [18]:
import pandas as pd
import numpy as np

# Semilla para reproducibilidad, hace que cada ves estare
#recibiendo los mismos valores aleatorios
np.random.seed(42)

# Cantidad de filas que quiero agregar que sea igual a
#la cantidad de mi otro dataset
n = 2900

# Las nuevas columnas con sus valores, dejando algunos en nan
#para poder trabajarlos
generos = ['Femenino', 'Masculino', np.nan]
tiene_mascota = ["Si", "No", np.nan]

# Paso a crear DataFrame con distribución de valores
df_ejemplo = pd.DataFrame({
    'Genero': np.random.choice(generos, size=n, p=[0.50, 0.40, 0.10]),
    'Tiene_mascota': np.random.choice(tiene_mascota, size=n, p=[0.5, 0.45, 0.05])
})

#print(df_ejemplo)

#Ahora genero un excel a partir del DataFrame creado anteriormente
df_ejemplo.to_excel('dataset_generado.xlsx', index=False)



Descripción general del proceso aplicado.
Problemas detectados y cómo se abordaron.
Preparación final del dataset y posibles usos posteriores.

Para este trabajo, descargué un dataset relacionado con los distintos tipos de personalidad. Para simular la integración de múltiples fuentes de información, creé un archivo Excel adicional con el mismo número de registros, al que añadí dos columnas nuevas con información complementaria.

Con el fin de poder unir ambas fuentes, incorporé una columna de ID en cada tabla y las mezclé utilizando el método merge() de pandas.

Posteriormente, apliqué diferentes métodos de exploración de datos para entender mejor la estructura del conjunto y detectar posibles valores faltantes. En esta etapa identifiqué valores nulos, los cuales traté reemplazándolos con el promedio en el caso de las columnas numéricas y con la moda en las columnas categóricas, asegurando así la consistencia del dataset.

También realicé una agrupación por columnas clave, renombré algunas de ellas para que fueran más breves y claras, y apliqué una función lambda para generar una nueva columna basada en transformaciones de datos numéricos existentes.

Como resultado, el dataset final quedó limpio y estructurado, permitiendo un análisis más preciso de las características promedio de cada tipo de personalidad. Este conjunto de datos puede utilizarse para explorar patrones generales de comportamiento.
